In [44]:
import sys
import numpy as np

x = np.array([-0.99, -0.98, -0.97, -0.96], dtype=np.float32)
# x = np.array([-0.99, -0.3, 0.3, 0.96], dtype=np.float32)

# symmetric case
a = -np.abs(x).max()
b = np.abs(x).max()

# a-symmetric case
# a = x.min()
# b = x.max()

q_a = -4
q_b = 4

s = (q_b - q_a)/(b - a)
q_c = (q_a + q_b)/2
c = (b + a)/2
zero_point = q_c - s*c


x_q = (s*x + zero_point).astype(np.int8)

x_approx = (x_q - zero_point)/s

mse = np.square(x_approx - x).sum()



In [1]:
import torch
import math
from utils import scalar_quantisation, scalar_dequantisation, get_tensor_memory_size

t = torch.rand((100, 100))
t_flat = t.flatten()

t_q = torch.zeros_like(t_flat).type(torch.int8)
chunk_size = 512
n_chunks = math.ceil(len(t_flat)/chunk_size)
scales = torch.zeros(n_chunks)
locations = torch.zeros(n_chunks)
for chunk_id in range(n_chunks):

    left = chunk_id * chunk_size
    right = min(len(t_flat), (chunk_id+1)*chunk_size) 

    t_q[left:right], scales[chunk_id], locations[chunk_id] = scalar_quantisation(t_flat[left:right])



In [2]:
get_tensor_memory_size(t_flat)

40000

In [4]:
get_tensor_memory_size(t_q) + get_tensor_memory_size(scales) + get_tensor_memory_size(locations)

10160

In [75]:
from utils import quantise_tensor_flat, dequantise_tensor_flat, compute_quantisation_mse,get_tensor_memory_size
import torch

t = torch.rand((100, 100))

t_q, scales, locations = quantise_tensor_flat(t, chunk_size=1024)
t_approx = dequantise_tensor_flat(t_q, scales, locations)

mse = compute_quantisation_mse(t.flatten(), t_approx)

print(f'Required memory: {get_tensor_memory_size(t)} vs {get_tensor_memory_size(t_q) + get_tensor_memory_size(scales) + get_tensor_memory_size(locations)}')

MSE: 0.2302183359861374
Norm: 57.57908248901367
RelativeMSE: 0.39983%
Required memory: 40000 vs 10080
